In [27]:
import gurobipy as gp
import itertools as it
import networkx as nx
import pandas as pd
import numpy as np
import math
import re

"""
implementation of solving profit maximization problem
author(s): 
yk796@cornell.edu
nd396@cornell.edu
"""

'\nimplementation of solving profit maximization problem\nauthor(s): \nyk796@cornell.edu\nnd396@cornell.edu\n'

In [28]:
# link to https://github.com/bstabler/TransportationNetworks/tree/master/SiouxFalls

network_df = pd.read_csv("../data/SiouxFalls/SiouxFalls_net.txt", sep='\t', comment=';')
node_df = pd.read_csv("../data/SiouxFalls/SiouxFalls_node.txt", sep='\t', comment=';')
od_df = pd.read_csv("../data/SiouxFalls/SiouxFalls_od.csv")

network_df = network_df[['init_node', 'term_node', 'capacity', 'length', 'free_flow_time', 'b',
       'power', 'speed', 'toll', 'link_type']]
network_df[['init_node', 'term_node']] = network_df[['init_node', 'term_node']].astype(int)
node_df = node_df[['Node', 'X', 'Y']]

print("Number of nodes:", len(node_df))
print("Number of links:", len(network_df))
print("Number of od pairs:", len(od_df))



Number of nodes: 24
Number of links: 76
Number of od pairs: 528


In [29]:
def generate_route_sets_link_elimination(graph, source, target, num_routes):
    route_sets = []
    for i in range(num_routes):
        path = nx.shortest_path(graph, source=source, target=target, weight='weight')
        if path not in route_sets:
            route_sets.append(path)
        
        if len(path) > 2:
            edge_to_remove = (path[1], path[2])
            original_weight = graph[edge_to_remove[0]][edge_to_remove[1]]['weight']
            graph[edge_to_remove[0]][edge_to_remove[1]]['weight'] = float('inf')
            
        else:
            break
        
    # Reset graph weights for future usage
    nx.set_edge_attributes(graph, original_weight, 'weight')
    
    return route_sets

# link penalty approach
def generate_route_sets_link_penalty(graph, source, target, num_routes, penalty_factor=1.5):
    route_sets = []
    for i in range(num_routes):
        path = nx.shortest_path(graph, source=source, target=target, weight='weight')
        if path not in route_sets:
            route_sets.append(path)
        
        for j in range(len(path) - 1):
            edge = (path[j], path[j+1])
            graph[edge[0]][edge[1]]['weight'] *= penalty_factor 
            
    # Reset graph weights for future usage
    nx.set_edge_attributes(graph, 1, 'weight')
    
    return route_sets


In [30]:
# input
n_nodes = len(node_df)


n_alternative = 1
n_routes = 3

p_sen = 1

# road_bpr_dict = {(row['init_node'], row['term_node']): lambda flow: row['free_flow_time']*(1+row['b']*(flow/row['capacity'])**row['power']) for index, row in network_df.iterrows()}

road_bpr_dict = {}
for index, row in network_df.iterrows():
    init_node, term_node = row['init_node'], row['term_node']
    free_flow_time, b, capacity, power = row['free_flow_time'], row['b'], row['capacity'], row['power']
    
    road_bpr_dict[(init_node, term_node)] = lambda flow, f=free_flow_time, b=b, c=capacity, p=power: f * (1 + b * (flow / c)) # 1 should be substitue with power

transit_bpr_dict = {(row['init_node'], row['term_node']): lambda flow: 100 for index, row in network_df.iterrows()}
 
bpr_func = {1: road_bpr_dict, 
            2: transit_bpr_dict}

nodes = node_df['Node'].to_list()
alternatives = list(range(1, n_alternative+1))
arcs = list(network_df[['init_node', 'term_node']].to_records(index=False))
#ods = list(it.permutations(nodes, 2))
# ods = [(id1+1, id2+1) for id1, o in enumerate(O_demand) for id2, d in enumerate(D_demand) if o>0 or d>0 if id1 != id2]

demand = {(row['O'], row['D']): row['Ton'] for index, row in od_df.iterrows()}
ods = list(demand.keys() )
road_link = [(int(row['init_node']), int(row['term_node']), row['length']) for _, row in network_df.iterrows()]


OD_route = {}
for od_pair_index in range(len(od_df)):
    i,j = od_df['O'].iloc[od_pair_index], od_df['D'].iloc[od_pair_index]
    G = nx.DiGraph()

    G.add_weighted_edges_from(road_link)
    
    route_sets = generate_route_sets_link_penalty(G, i, j, n_routes) 
    

    OD_route[(i,j)] = route_sets
    

T = {key:10 for key in list(it.product(ods, alternatives))}
ASC = {key:0 for key in list(it.product(ods, alternatives))}


def create_route(od):
    (o, d) = od
    # find all possible routes
    return (o, d)
    

def indicator(arc, route): 
    '''
    To check if an arc is in route
    '''
    # Check if arc is a tuple and has 2 elements
    if not isinstance(arc, tuple) or len(arc) != 2:
        raise ValueError("Arc must be a tuple with 2 elements")

    # Iterate through the route and check each pair
    for i in range(len(route) - 1):
        if route[i] == arc[0] and route[i+1] == arc[1]:
            return True
    return False




In [31]:
def profit_maximization(n_nodes, arcs, routes, n_alternative, ods, demand, T, ASC, bpr_func):
    eps = 1e-3

    m = gp.Model()
    m.Params.NonConvex = 2 
    m.Params.DualReductions = 0 # to determine if the model is infeasible or unbounded
    # m.Params.OutputFlag = 0
    # m.Params.NonConvex = 2 # because of profit_extracting_log term is concave, we need to tell gurobi
    # that this is not a concave model, although it is. 
    #m.Params.MIPGap = 0


    m._T = T
    m._ASC = ASC
    m._bpr_func = bpr_func
    m._nodes = list(range(1, n_nodes+1))
    m._alternatives = list(range(1, n_alternative+1))
    m._arcs = [tuple(int(a) for a in arc) for arc in arcs]
    m._ods = ods #[(id1+1, id2+1) for id1, o in enumerate(O_demand) for id2, d in enumerate(D_demand) if o>0 or d>0 if id1 != id2]
    #m._routes = routes# {key:create_route(key) for key in m._ods}

    m._routes = routes

    for (o, d) in m._ods:
        # Convert to list of tuples with native Python types
        m._routes[(o, d)] = [tuple(int(r) for r in route) for route in routes[(o, d)]]

    m._theta_vars = m.addVars(list(it.product(m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=eps, ub=1, name='theta')

    m._y_vars = m.addVars(list(it.product(m._arcs, m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=0, name='y') # in fully connected graph, y=z
    
    # for yvar in m._y_vars.values():
    #     yvar.start = 100

    
    #############
    ## This part is problematic for multiple ODs. It overwrites previous ODs. 
    # for (o,d) in m._ods:
    #     m._z_vars = m.addVars(list(it.product(m._routes[(o,d)], m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=0, name='z') # in fully connected graph, y=z
    # ##############
    
    z_vars_list = []
    for (o, d) in m._ods:
        # Create a list of tuples with native Python types for each (o, d) pair and append it to z_vars_list
        z_vars_list.extend([(tuple(int(r) for r in route), int(alt))
                            for route, alt in it.product(m._routes[(o, d)], m._alternatives)])
        
    m._z_vars = m.addVars(z_vars_list, vtype = gp.GRB.CONTINUOUS, lb = 0, name = 'z')
    
    
    
    
    
    # create auxiliary variables to deal with non-linear objective function

#     m._fvars = m.addVars(list(it.product(m._alternatives, m._arcs)), vtype=gp.GRB.CONTINUOUS, lb=0, name='f') 
    m._fvars = m.addVars(m._arcs, vtype = gp.GRB.CONTINUOUS, lb = 0, name = 'f')
    
    
    # # Dictionary for initial values
    # initial_values = {(1, 2): 100, (2, 4): 100, (1, 3):100, (3, 4): 100}

    # # Loop to set initial values
    # for arc, init_val in initial_values.items():
    #     m._fvars[arc].Start = init_val
    
    #m._ln_theta_vars = m.addVars(list(it.product(m._ods, m._alternatives)), lb = -float('inf'), ub = 0, vtype=gp.GRB.CONTINUOUS, name='ln_theta')
    m._theta_lntheta_vars = m.addVars(list(it.product(m._ods, m._alternatives)), lb = -float('inf'), ub = 0, vtype=gp.GRB.CONTINUOUS, name='theta_ln_theta') #define theta * ln(theta)
    m._congest_tt = m.addVars(list(it.product(m._ods, m._alternatives)), lb = 0, vtype=gp.GRB.CONTINUOUS, name='congested_travel_time') 


    m._ind = m.addVars(m._ods, vtype=gp.GRB.BINARY, name="ind")
    m._profit_extracting = m.addVars(m._ods, vtype=gp.GRB.CONTINUOUS, lb= 0, ub=1, name='extracting')
    m._profit_extracting_log = m.addVars(m._ods, vtype=gp.GRB.CONTINUOUS, lb = -float('inf'), ub=0, name='extracting_log')
    m._profit_extracting_term = m.addVars(list(it.product(m._ods, m._alternatives)), lb = -float('inf'), ub = 0, vtype=gp.GRB.CONTINUOUS, name='extracting_term')
    
    #############
    ## F and G need to be distinguished by alternative j?
    #m._F = m.addVars(list(it.product(m._alternatives, m._arcs)), vtype=gp.GRB.CONTINUOUS, name='F')
    #m._G = m.addVars(list(it.product(m._alternatives, m._arcs)), vtype=gp.GRB.CONTINUOUS, name='G')
    #############
    
    m._F = m.addVars(m._arcs, vtype = gp.GRB.CONTINUOUS, name = 'F')
    m._G = m.addVars(m._arcs, vtype = gp.GRB.CONTINUOUS, name = 'G')
    

    """add constraints"""
    # relationship between theta and z
    for j in m._alternatives:
        for (s, t) in m._ods:
            lhs = gp.quicksum(m._z_vars[r, j] for r in m._routes[(s, t)])
            rhs = m._theta_vars[(s, t), j]
            m.addConstr(lhs == rhs, name = "constraint Q (a)")
            
         
    
    # sum of theta's <= 1
    for (s, t) in m._ods:
        lhs = gp.quicksum(m._theta_vars[(s, t), j] for j in m._alternatives)
        rhs = 1 #- eps
        m.addConstr(lhs <= rhs, name = "constraint Q (b)") 
    
      
        
    # load on arc needed  
    for j in m._alternatives:
        for a in m._arcs:
            for (s, t) in m._ods:
                m.addConstr(gp.quicksum([demand[s,t] * m._z_vars[r, j] for r in m._routes[(s, t)] if indicator(a, r)]) == m._y_vars[a, (s, t), j], name = "constraint Q (c)") 

                
                

    # relationship to f_a
#     for j in alternatives:
#         for a in m._arcs:
#             lhs = m._fvars[j,a]
#             rhs = gp.quicksum(m._y_vars[a, (s, t), j] for (s, t) in m._ods)
#             m.addConstr(lhs == rhs, name = "equation Q (d)")
            
            
    for a in m._arcs:
        lhs = m._fvars[a]
        rhs = gp.quicksum(m._y_vars[a, (s, t), j] for (s, t) in m._ods for j in m._alternatives)
        m.addConstr(lhs == rhs, name = "equation Q (d)")
    

    # flow conservation
    for j in m._alternatives:
        for v in m._nodes:
            if (v != s) and (v != t):
                lhs = gp.quicksum(m._y_vars[(a, b) , (s, t), j] for (a,b) in m._arcs if v == a)
                rhs = gp.quicksum(m._y_vars[(a, b) , (s, t), j] for (a,b) in m._arcs if v == b)
                m.addConstr(lhs == rhs, name = "constraint Q (e)")


    bins = 100
    xs = [1/bins*i for i in range(bins+1)]
    ys = [p*math.log(p) if p != 0 else 0 for p in xs]
    # objective function
    for j in m._alternatives:
        for (s, t) in m._ods:
            #m.addConstr(m._theta_vars[(s, t), j] * m._inv_theta_vars[(s, t), j] == 1, name = "inv_theta") 
            #m.addGenConstrLog(m._theta_vars[(s, t), j], m._ln_theta_vars[(s, t), j], name = "ln_theta") 
            m.addGenConstrPWL(m._theta_vars[(s, t), j], m._theta_lntheta_vars[(s, t), j], xs, ys, "pwl")

    # constraints for profit extracting term
    for (s, t) in m._ods:
        m.addConstr(m._profit_extracting[s,t] == 1 - gp.quicksum(m._theta_vars[(s, t), j] for j in [1]), name ='extract') # add except transit mode
        m.addGenConstrLog(m._profit_extracting[s,t], m._profit_extracting_log[s,t], name = "ln_profit")
        for j in m._alternatives:
            m.addConstr(m._profit_extracting_term[(s, t), j] == m._theta_vars[(s, t), j] * m._profit_extracting_log[s,t])
            
#     for j in m._alternatives:
#         for (s, t) in m._ods:
#             #m.addConstr(m._congest_tt[(s, t), j] == gp.quicksum(gp.quicksum(m._z_vars[r, j]*m._F[a] for r in m._R[(s, t)] if indicator(a, r)) for a in m._arcs))

#             m.addConstr(m._congest_tt[(s, t), j] == gp.quicksum(gp.quicksum(m._z_vars[r, j]*m._F[j, a] for a in m._arcs if indicator(a, r)) for r in m._routes[(s, t)]))



    for (s, t) in m._ods:
        m.addConstr(m._congest_tt[(s, t), j] == gp.quicksum(gp.quicksum(m._z_vars[r, j]*m._F[a] for a in m._arcs if indicator(a, r)) for r in m._routes[(s, t)] for j in m._alternatives))


            
            
            
    # Define F and G
#     for j in alternatives:
#         for a in m._arcs:
#             lhs = m._F[j,a]
#             rhs = m._bpr_func[j][a](m._fvars[j,a])
#             m.addConstr(lhs == rhs, name = "F_function")

#             lhs = m._G[j,a]
#             rhs = m._bpr_func[j][a](m._fvars[j,a])
#             m.addConstr(lhs == rhs, name = "G_function")
            

    for a in m._arcs:
        lhs = m._F[a]
        rhs = m._bpr_func[1][a](m._fvars[a])
        m.addConstr(lhs == rhs, name = "F_function")

        lhs = m._G[a]
        rhs = m._bpr_func[1][a](m._fvars[a])
        m.addConstr(lhs == rhs, name = "G_function")
            

    # # define objective function
    # m.setObjective(gp.quicksum(D[s,t]* gp.quicksum(m._theta_vars[(s, t), j] * (m._T[(s, t), j] - m._ASC[(s, t), j]) for j in m._alternatives) for (s, t) in m._ods) # objective function (a)
    #                )
    
    obj_util = gp.quicksum(demand[s,t]/p_sen * gp.quicksum(m._theta_vars[(s, t), j] * (m._T[(s, t), j] - m._ASC[(s, t), j]) for j in m._alternatives) for (s, t) in m._ods) # objective function (A)
    obj_congest = gp.quicksum(demand[s,t]/p_sen * m._congest_tt[(s, t), j] for j in m._alternatives for (s, t) in m._ods)
    obj_entropy = gp.quicksum(demand[s,t]/p_sen * gp.quicksum(m._theta_lntheta_vars[(s, t), j] for j in m._alternatives) for (s, t) in m._ods) # objective function (B)
    obj_profit_extracting = - gp.quicksum(demand[s,t]/p_sen * gp.quicksum(m._profit_extracting_term[(s, t), j] for j in m._alternatives) for (s, t) in m._ods) # objective function (C) 
    obj_oper_cost = gp.quicksum(gp.quicksum(m._G[a]* m._y_vars[a , (s, t), j] for a in m._arcs) for j in m._alternatives for (s, t) in m._ods) # objective function (D)
    # define objective function
    m.setObjective(obj_util + obj_congest + obj_entropy + obj_profit_extracting) # +  + obj_oper_cost #  + 

    m.update()
    m.optimize()

    # m.computeIIS() # this helps us to identify constraints that are responsible to make the model infeasible.
    # m.write("model.ilp")

    for v in m.getVars():
        print(f"{v.VarName} = {v.X}")


    if m.Status == 3:
        return None, None
    else:
        return m
    


result = profit_maximization(n_nodes, arcs, OD_route, n_alternative, ods, demand, T, ASC, bpr_func)



Set parameter NonConvex to value 2
Set parameter DualReductions to value 0


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) Gold 6244 CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 41962 rows, 45211 columns and 88498 nonzeros
Model fingerprint: 0x9c2814d1
Model has 1056 quadratic constraints
Model has 1056 general constraints
Variable types: 44683 continuous, 528 integer (528 binary)
Coefficient statistics:
  Matrix range     [1e-05, 4e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+02, 4e+04]
  Bounds range     [1e-03, 1e+00]
  RHS range        [1e+00, 1e+01]
  PWLCon x range   [1e-02, 1e+00]
  PWLCon y range   [1e-02, 4e-01]
Presolve added 0 rows and 66280 columns
Presolve removed 38814 rows and 0 columns
Presolve time: 0.23s
Presolved: 23562 rows, 115888 columns, 330755 nonzeros
Presolved model has 1056 SOS constraint(s)
Presolved model has 4985 bilinear constrain

In [32]:
# open a file in write mode
with open("output.txt", "w") as file:
    for v in result.getVars():
        # Write each line to the file
        file.write(f"{v.VarName} = {v.X}\n")

# TODO: calculating profit
# 

In [33]:
V_n = {key:0 for key in ods}

In [34]:
def extract_result(result):
    # extract optimal value for theta, congest_tt
    theta_result = {}
    congest_tt_result = {}
    for v in result.getVars():
        if v.VarName[:6] == 'theta[':
            item = re.findall("\[(.*?)\]", v.VarName)[0] 
            first, second, third = item.split(',')       
            theta_result[(int(first[1]), int(second[1])), int(third)] = v.X
        if v.VarName[:22] == 'congested_travel_time[':
            item = re.findall("\[(.*?)\]", v.VarName)[0] 
            first, second, third = item.split(',')       
            congest_tt_result[(int(first[1]), int(second[1])), int(third)] = v.X
                        
    print('theta_result: ', theta_result)
    print('congest_tt_result: ', congest_tt_result)
    
    return theta_result, congest_tt_result
    

In [35]:
def prob_to_price(theta_result, congest_tt_result): 
     
    if len(theta_result) == 0:
        raise Exception("no theta values")
      
    # calculate theta^n_st for each pair st
    theta_n = {key: 1 for key in ods}
    for key, value in theta_result.items():
        theta_n[key[0][0], key[0][1]] -= value
        
    print('theta_n: ', theta_n)
        
    # calculate l^j_st for each alternative j and pair st
    l = {}
    for key, value in theta_result.items():
        l[key] = congest_tt_result[key] / theta_result[key]       
    print('l: ', l)
    
    
    
    # calculate pi^j_st for each alternative j and pair st
    pi = {}   
    for key, value in theta_result.items():
        od = (key[0][0], key[0][1])
        pi[key] = (-1/p_sen) * (math.log(value) - math.log(theta_n[od]) + V_n[od] - ASC[key] + T[key] + l[key])       
    print('pi: ', pi)
    
    
    return pi
        
    
    
    
    
    
    

    

In [36]:
def rev(demand, theta_dict, pi):
    revenue = 0
    for key, value in theta_dict.items():
        od = (key[0][0], key[0][1])
        revenue += demand[od] * value * pi[key]
    
    print('revenue: ', revenue)
    
    return revenue

In [37]:
theta_result, congest_tt_result = extract_result(result)

theta_result:  {((2, 1), 1): 0.0010000000006089237, ((3, 1), 1): 0.001000000000408993, ((4, 1), 1): 0.00100000000022242, ((5, 1), 1): 0.0010000000004892426, ((6, 1), 1): 0.0010000000003040365, ((7, 1), 1): 0.001000000000172895, ((8, 1), 1): 0.001000000000098485, ((9, 1), 1): 0.0010000000001516684, ((1, 1), 1): 0.0010000000002680746, ((1, 2), 1): 0.0010000000006089244, ((3, 2), 1): 0.0010000000005427766, ((4, 2), 1): 0.0010000000002444126, ((5, 2), 1): 0.0010000000004665014, ((6, 2), 1): 0.0010000000003929537, ((7, 2), 1): 0.0010000000004894822, ((8, 2), 1): 0.0010000000002129779, ((9, 2), 1): 0.0010000000002224811, ((2, 2), 1): 0.001000000000192443, ((1, 3), 1): 0.001000000000408993, ((2, 3), 1): 0.0010000000005427766, ((4, 3), 1): 0.0010000000005313838, ((5, 3), 1): 0.0010000000017504572, ((6, 3), 1): 0.0010000000002157684, ((7, 3), 1): 0.0010000000004894353, ((8, 3), 1): 0.001000000000286638, ((9, 3), 1): 0.0010000000006091213, ((1, 4), 1): 0.00100000000022242, ((2, 4), 1): 0.0010000

In [38]:
pi = prob_to_price(theta_result, congest_tt_result)

KeyError: (1, 1)

In [ ]:
total_rev = rev(demand, theta_result, pi)

revenue:  -4.620648827222043
